# Оптимизация JOIN

## Подготовка

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, explode, min, max
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession \
  .builder \
  .appName("Joins Optimizations") \
  .master("local[4]") \
  .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
  .getOrCreate()

sc = spark.sparkContext

In [ ]:
! cd /tmp && rm -rf steam && rm -rf /tmp/spark-warehouse && unzip ~/notebooks/steam.zip && cd -

In [ ]:
games = spark.read.parquet("/tmp/steam/games.parquet")
details = spark.read.parquet("/tmp/steam/details.parquet")
tags = spark.read.parquet("/tmp/steam/tags.parquet")
recomendations = spark.read.parquet("/tmp/steam/recomendations.parquet")
users = spark.read.parquet("/tmp/steam/users.parquet")

In [ ]:
games.printSchema()

In [ ]:
recomendations.printSchema()

In [ ]:
users.printSchema()

In [ ]:
details.printSchema()

In [ ]:
tags.printSchema()

## Оптимизация через анализ потребления ресурсов

In [ ]:
result = details \
  .join(tags, "tag") \
  .join(games, "app_id")
result.count()
result.explain()

In [ ]:
result = details \
  .join(games, "app_id") \
  .join(tags, "tag")
result.count()
result.explain()

## Оптимизация на базе стомости

In [ ]:
spark.conf.set("spark.sql.cbo.enabled", True)
spark.conf.set("spark.sql.cbo.joinReorder.enabled", True)
spark.conf.set("spark.sql.statistics.histogram.enabled", True)
spark.conf.set("spark.sql.statistics.size.autoUpdate.enabled", True)

In [ ]:
spark.sql("DROP TABLE IF EXISTS games")
spark.sql("DROP TABLE IF EXISTS details")
spark.sql("DROP TABLE IF EXISTS tags")

In [ ]:
games.write.mode("overwrite").saveAsTable("games")
details.write.mode("overwrite").saveAsTable("details")
tags.write.mode("overwrite").saveAsTable("tags")

In [ ]:
spark.sql("ANALYZE TABLE games COMPUTE STATISTICS FOR ALL COLUMNS")

In [ ]:
spark.sql("DESC EXTENDED games app_id") \
  .repartition(1) \
  .withColumn("id", F.monotonically_increasing_id()) \
  .where(col("id").between(0, 21) | col("id").between(250, 270)) \
  .show(40, False)

In [ ]:
games \
  .select(
    min(col("app_id")),
    max(col("app_id"))
  ) \
  .show()

In [ ]:
spark.sql("ANALYZE TABLE details COMPUTE STATISTICS FOR ALL COLUMNS")
spark.sql("ANALYZE TABLE tags COMPUTE STATISTICS FOR ALL COLUMNS")

In [ ]:
res = spark.sql("""
  SELECT *
    FROM details
    JOIN tags USING (tag)
    JOIN games USING (app_id)
""")
res.count()
res.explain()

In [ ]:
res = spark.sql("""
  SELECT *
    FROM games
    JOIN details USING (app_id)
    JOIN tags USING (tag)
""")
res.count()
res.explain()

## Оптимизация при помощи hint инструкций

### Spark DSL

In [ ]:
users.join(recomendations, "user_id").explain()

In [ ]:
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 2000)

In [ ]:
users.join(recomendations.hint("SHUFFLE_HASH"), "user_id").explain()

In [ ]:
users.hint("BROADCAST").join(recomendations, "user_id").explain()

### Spark SQL

In [ ]:
users.createOrReplaceTempView("users")
recomendations.createOrReplaceTempView("recomendations")

In [ ]:
spark.sql("SELECT * FROM users JOIN recomendations USING (user_id)").explain()

In [ ]:
spark.sql("SELECT /*+ SHUFFLE_HASH(r) */ * FROM users u JOIN recomendations r USING (user_id)").explain()

In [ ]:
spark.sql("SELECT /*+ BROADCAST(u) */ * FROM users u JOIN recomendations r USING (user_id)").explain()

In [ ]:
spark.sql("SELECT /*+ SHUFFLE_REPLICATE_NL(u) */ * FROM users u JOIN recomendations r USING (user_id)").explain()

## Оптимизация неравно распределенных партиций

### Воспользоваться AQE

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 200)
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", False)

In [ ]:
result = (
  recomendations
    .where(col("user_id").isin(11, 18, 32, 36, 42, 56, 60, 66, 67, 68, 75, 79, 83, 89, 104, 105, 110, 124, 131, 134, 3632140))
    .repartition(21, "user_id")
    .join(users, "user_id") 
)
result.explain()

In [ ]:
result.show()

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", True)
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", True)
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionFactor", 5)
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", 10)

In [ ]:
result = (
  recomendations
    .where(col("user_id").isin(11, 18, 32, 36, 42, 56, 60, 66, 67, 68, 75, 79, 83, 89, 104, 105, 110, 124, 131, 134, 3632140))
    .repartition(21, "user_id")
    .join(users, "user_id") 
)
result.explain()

In [ ]:
result.show()

In [ ]:
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionFactor", 5)
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", 268435456)

### Ручная подготовка ключа

In [ ]:
recomendations_salted = recomendations \
  .where(col("user_id").isin(11, 18, 32, 36, 42, 56, 60, 66, 67, 68, 75, 79, 83, 89, 104, 105, 110, 124, 131, 134, 3632140)) \
  .withColumn(
    "user_id_salted",
    F.concat(
        col("user_id"),
        F.lit("_"),
        F.floor(F.rand() * 3)
    )
  )

In [ ]:
users_salted = users \
  .withColumn(
    "user_id_salted",
    F.explode(
      F.array(
        [F.concat(col("user_id"), F.lit("_"), F.lit(x))
          for x in range(3)
        ]
      )
    )
  )

In [ ]:
salted_join_df = recomendations_salted \
  .join(users_salted, "user_id_salted") \
  .drop(users_salted.user_id) \
  .drop(users_salted.user_id_salted)

In [ ]:
salted_join_df.show(5, False)

## Theta join

In [ ]:
fav_genres = sorted(["Action", "Comedy", "Horror", "Thriller", "Sci-fi"])

In [ ]:
fav_genres_df = tags.where(col("tag").isin(fav_genres))
details_df = details.where(col("tag").isin(fav_genres))

In [ ]:
details_df.join(
    fav_genres_df,
    details_df.tag < fav_genres_df.tag
).count()

In [ ]:
fav_genres_df.createOrReplaceTempView("fav_genres")
spark.sql("""
SELECT *
  FROM details d
  JOIN fav_genres g
    ON (d.tag < g.tag)
 WHERE d.tag in (select tag from fav_genres)
"""
).count()

In [ ]:
spark.sql("""
SELECT *
  FROM details d
  JOIN fav_genres g
    ON (CASE 
          WHEN g.tag == "Comedy" THEN d.tag == "Action"
          WHEN g.tag == "Horror" THEN d.tag == "Action" OR d.tag == "Comedy"
          WHEN g.tag == "Sci-fi" THEN d.tag == "Action" OR d.tag == "Comedy" OR d.tag == "Horror"
          WHEN g.tag == "Thriller" THEN d.tag == "Action" OR d.tag == "Comedy" OR d.tag == "Horror" OR d.tag == "Sci-fi"
          ELSE false
        END
       )
"""
).count()